## Using conterfactual regret minimization to find Nash equilibrium in Rock, Paper, Scissors
Refs:  
- http://modelai.gettysburg.edu/2013/cfr/cfr.pdf
- https://github.com/RichardKelley/cfr/blob/master/rps.py

### Algorithm
- Rock, Paper and Scissors are encoded as 0,1,2
- Strategies are computed from positive regret (If regret <= 0, uniform strategy)
- Add strategy to strategy profile sum
- Get actions for each strategy
- Compute regrets for actions
- Use regrets to compute new strategies and repeat...
- Compute average strategy profile at the end

Note: This execution is for a mixed strategy for the opponent so we can compute the Nash equilibruim

In [1]:
import numpy as np
import time

start = time.time()

def get_strategy(regret_sum):
  strategy = np.array([i if i > 0 else 0 for i in regret_sum])  # Consider only positive regret values
  if(sum(strategy) > 0):  strategy = np.array([i/sum(strategy) for i in strategy])  # Case check for [0,0,0]
  else: strategy = np.array([1/3,1/3,1/3])
  return(strategy)

def get_action(strategy):
  # Choosing action based on probability distribution
  return(np.random.choice(np.arange(num_actions),1,p = strategy)[0])  
  

def get_utility(action):
  # Utility is always cyclic in 0,1,-1 for RPS in that order
  util = np.array([0.0,0.0,0.0]) 
  util[(opp_action+1)%3] = 1  
  util[(opp_action+2)%3] = -1
  return(util)

rock, paper, scissors = 0,1,2
num_actions = 3

my_strategy_sum = np.array([0.0,0.0,0.0])
my_regret_sum = np.array([0.0,0.0,0.0])

opp_strategy_sum = np.array([0.0,0.0,0.0])
opp_regret_sum = np.array([0.0,0.0,0.0])

for i in range(100000):
  my_strategy = get_strategy(my_regret_sum)
  opp_strategy = get_strategy(opp_regret_sum) 
  # opp_strategy can be a fixed p-distrib array if opponent takes a pure strategy instead of mixed
  
  # Summing all strategies to be averaged at the end
  my_strategy_sum += my_strategy
  opp_strategy_sum += opp_strategy
  # Choosing actions based on strategies
  my_action = get_action(my_strategy)
  opp_action = get_action(opp_strategy)

  util_for_my_action = get_utility(my_action)  #Utility for choosing my action vs. all actions
  util_for_opp_action = get_utility(opp_action)  #Utility for choosing opponent's action vs. all actions

  my_regret_sum = util_for_opp_action - util_for_opp_action[my_action]
  opp_regret_sum = util_for_my_action - util_for_my_action[opp_action]

print("Nash Equilibrium\nPlayer 1 (me):\t",[round(i/sum(my_strategy_sum),4) for i in my_strategy_sum], "\nPlayer 2 (opp):\t",[round(i/sum(opp_strategy_sum),4) for i in opp_strategy_sum])
print("Computed in",round(time.time()-start,2),"s")

Nash Equilibrium
Player 1 (me):	 [0.3333, 0.3333, 0.3333] 
Player 2 (opp):	 [0.3333, 0.3333, 0.3333]
Computed in 7.12 s
